<a href="https://colab.research.google.com/github/sokrypton/ColabFold/blob/main/beta/ESMFold_api.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**ESMFold API**
for more details see: [Github](https://github.com/facebookresearch/esm/tree/main/esm), [Preprint](https://www.biorxiv.org/content/10.1101/2022.07.20.500902v1)

#### **Tips and Instructions**
- click the little ▶ play icon to the left of each cell below.

#### **Limitations**
- max total length=400
- for lengths > 400, try our [colab notebook](https://colab.research.google.com/github/sokrypton/ColabFold/blob/main/ESMFold.ipynb)
---

*credit: [@drchristhorpe](https://twitter.com/drchristhorpe)*

In [ ]:
#@title run **ESMFold**
import os, time, re, requests
if not os.path.isfile("colabfold.py"):
  os.system("pip -q install py3Dmol")
  os.system("wget -qnc https://raw.githubusercontent.com/sokrypton/ColabFold/main/beta/colabfold.py")
  os.system("mkdir -p tmp")

import colabfold as cf

def predict(sequence):
  esmfold_api_url = 'https://api.esmatlas.com/foldSequence/v1/pdb/'
  r = requests.post(esmfold_api_url, data=sequence)
  while r.status_code != 200:
    time.sleep(5)
    r = requests.post(esmfold_api_url, data=sequence)
  structure = r.text
  return structure

sequence = "GWSTELEKHREELKEFLKKEGITNVEIRIDNGRLEVRVEGGTERLKRFLEELRQKLEKKGYTVDIKIE" #@param {type:"string"}
sequence = re.sub("[^A-Z]", "", sequence.upper())
#assert len(sequence) <= 400, "error: max length supported is 400"
hash = cf.get_hash(sequence)[:5]
pdb_filename = f"tmp/prediction_{hash}.pdb"
if not os.path.isfile(pdb_filename):
  pdb_str = predict(sequence)
  with open(pdb_filename,"w") as out:
    out.write(pdb_str)

color = "confidence" #@param ["confidence", "rainbow"]
if color == "confidence": color = "lDDT"
show_sidechains = False #@param {type:"boolean"}
show_mainchains = False #@param {type:"boolean"}
v = cf.show_pdb(pdb_filename,
                show_sidechains,
                show_mainchains, color,
                color_HP=True,
                size=(800,480),
                vmin=0.5, vmax=0.9)       
v.setHoverable({}, True,
               '''function(atom,viewer,event,container){if(!atom.label){atom.label=viewer.addLabel("      "+atom.resn+":"+atom.resi,{position:atom,backgroundColor:'mintcream',fontColor:'black'});}}''',
               '''function(atom,viewer){if(atom.label){viewer.removeLabel(atom.label);delete atom.label;}}''')
v.show()           
if color == "lDDT":
  cf.plot_plddt_legend().show()

In [ ]:
#@title download prediction
from google.colab import files
files.download(pdb_filename)